In [5]:
import timeatlas as ta
from timeatlas import TimeSeries, Metadata
from timeatlas import models
from pandas import Series, DatetimeIndex
import pandas as pd
import os
from pathlib import Path
import fbprophet

---
# Models

How to create models with TimeAtlas? At first let's create a TimeSeries.

In [7]:
index = DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04'])
my_series = Series([0.4, 1.0, 0.7, 0.6], index=index)
ts = TimeSeries(my_series)
ts

## Grammar

Some definitions:

- `time series` : self-explanatory, i.e. the TimeSeries object
- `horizon` : the duration to predict after the last value of the time series
- `frequency`: the number of values per unit of time. Usually, the frequency is given in Pandas offset aliases (https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases)

```
                                horizon
   |-------------------------|- - - - - -|
                             |||||||||||||
           time series         frequency

```

In [ ]:
m = models.Prophet()

In [ ]:
m.fit(ts)

In [ ]:
Y_hat = m.predict('4 days')

In [ ]:
m.X_train.plot()
Y_hat.plot()

# NN-Models

In [2]:
from pandas import Series, DatetimeIndex
from torch.utils.data import DataLoader
from timeatlas import TimeSeriesDataset, TimeSeries
from timeatlas.models.NN import TimeSeriesClassificationDataset

### MLP Classification

In [3]:
index = DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04'])
my_series = Series([0.4, 1.0, 0.7, 0.6], index=index)
ts = TimeSeries(my_series)
ts.label = 1
tsd = TimeSeriesDataset(data=[ts, ts])

In [4]:
dataset = TimeSeriesClassificationDataset(tsd)
dataloader = DataLoader(dataset, batch_size=1)

dataset.data

array([[0.4, 1. , 0.7, 0.6],
       [0.4, 1. , 0.7, 0.6]])

In [5]:
for i in dataloader:
    print(i)

[tensor([[0.4000, 1.0000, 0.7000, 0.6000]], dtype=torch.float64), tensor([1])]
[tensor([[0.4000, 1.0000, 0.7000, 0.6000]], dtype=torch.float64), tensor([1])]


### LSTM Prediction

In [1]:
from pandas import Series, DatetimeIndex
from torch.utils.data import DataLoader
from timeatlas import TimeSeriesDataset, TimeSeries
from timeatlas.models import NeuralNetwork
from timeatlas.models.NN.models.LSTM import LSTMPrediction
from timeatlas.models.NN import TimeSeriesPredictionDataset

import torch.nn as nn
import torch

import numpy as np
import datetime

In [2]:
index = DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04'])
my_series = Series([0.4, 1.0, 0.7, 0.6], index=index)
ts = TimeSeries(my_series)
tsd = TimeSeriesDataset(data=[ts, ts])

In [ ]:
dataset = TimeSeriesPredictionDataset(data=tsd, n=3)
dataloader = DataLoader(dataset, batch_size=2)
dataset.data

In [15]:
model = LSTMPrediction(n_features=1, n_hidden=32, seq_len=3, n_layers=2, dropout=0.1).double()
NN = NeuralNetwork(model=model, epochs=100, optimizer=torch.optim.Adam, loss_funciton=nn.MSELoss())

In [16]:
model

LSTMPrediction(
  (lstm): LSTM(1, 32, num_layers=2, dropout=0.1)
  (linear): Linear(in_features=32, out_features=1, bias=True)
)

In [17]:
NN.fit(dataloader)

/Users/lorenz/.local/share/virtualenvs/timeatlas-kzsAXcI_/lib/python3.7/site-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch:   1 loss: 0.11255116
epoch:  26 loss: 0.00199771
epoch:  51 loss: 0.00011119
epoch:  76 loss: 0.00004447
epoch:  99 loss: 0.0000450084
